In [10]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [2]:
driver_path = 'C:/Users/MY/Downloads/chromedriver.exe'

In [3]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [ ]:
def biz_khan(query,driver_path,page):
    domain = '문화일보'
    
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    head_list = []
    data_list = []
    
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path)
        driver = webdriver.Chrome(executable_path = driver_path)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain = '%EB%89%B4%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1021&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        
        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
                
        except:
            print('기사가 없')
            driver.close()
            return 0
        driver.close()
        
        
        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path)  # for Windows
            try:
                driver.get(url)
                time.sleep(1)
            
            except:
                print('link error')
                continue
                # try문 안에서 각자 언론사에 맞게 수정
                
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.find('span', class_ = 'title').text)
                data = soup.select('#NewsAdContent')[0].text
                data_list.append(data)
            
            except:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.find('h3', id = 'articleTitle').text)
                data = soup.select('#articleBodyContents')[0].text
                data_list.append(data)
            driver.close()
        current_page+=10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

biz = biz_khan('풀무원 바른먹거리',driver_path,2)
biz

In [6]:
biz.to_csv('문화일보.csv',index=None)

In [7]:
data1 = pd.read_csv('문화일보.csv')

In [8]:
data1

,언론사,제목,내용
0,문화일보,"풀무원, 어린이 바른 먹거리 교육 ‘블렌디드 교육’ 실시",\n수업 집중도 높아 전국으로 확대 방침\n\n풀무원은 어린이 바른 먹거리 교육을 ...
1,문화일보,"풀무원 올가홀푸드, 농림부 장관상 수상",\n친환경 농산물 유통 및 소비 촉진에 공로\n\n풀무원은 계열인 올가홀푸드 강병규...
2,문화일보,풀무원 ‘바른먹거리 캠페인’ 확대,\n\t\t\t\t 풀무원의 ‘바른먹거리 캠페인’이 대표적인 어린이 먹거리 조기 ...
3,문화일보,풀무원 ‘바른먹거리 캠페인’ 하반기 교육 대폭 확대실시,\n\t\t\t\t 풀무원이 어린이들에게 바른먹거리 선택법을 알려주는 ‘바른먹거리...
4,문화일보,풀무원 ‘바른먹거리 교육’ 1만2561명 몰려,\n\t\t\t\t 식품기업인 풀무원이 소비자단체와 함께 시행하고 있는 ‘바른먹거...
5,문화일보,‘풀무원 재단’ 공식 출범… 바른먹거리·나눔 추진,\n\t\t\t\t 풀무원이 기업의 사회적 책임(CSR) 활동을 확대·강화하기 위...
6,문화일보,"풀무원, 독립투쟁정신 기리는 영석상 받아",\n1회 우당상엔 김성수 前대주교\n\n“독립운동에 전 재산과 생명을 바친 영석 이...
7,문화일보,"고령사회, 어르신 바른 식생활 시급, 풀무원 개선사업 펼친다",\n\t\t\t\t 풀무원이 고령자를 위한 바른 먹거리 교육에 나서 주목을 받고 ...
8,문화일보,풀무원은 ‘먹거리 지킴이’… 식습관 개선 캠페인 3년째,\n\t\t\t\t 풀무원이 어린이 식생활 습관 개선을 위해 3년째 진행해 온 ‘...
9,문화일보,"중국 진출 만 7년 풀무원, 식품사업 협력 강화","\n\t\t\t\t 중국 진출 만 7년을 맞는 풀무원이 베이징(北京), 상하이(上..."


In [12]:
query = '삼성전자 슬로건'
domain = '문화일보'
driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
url_domain = 'https://search.naver.com/search.naver?where=news&query='
news_domain ='%EB%89%B4%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1021&nso=&is_sug_officeid=0' 
url = url_domain + query + news_domain
driver.get(url)
soup = BeautifulSoup(driver.page_source,'html.parser')
news_list = soup.select('#main_pack > section > div > div.group_news > ul')
url_list = []
news_list = news_list[0].find_all('li')
for url in news_list:
    url_list.append(str(url.find_all('a')[0].get('data-url')))
driver.close()

In [16]:
driver = webdriver.Chrome(executable_path=driver_path)  # for Windows
driver.get(url_list[6])

In [26]:
driver.close()

In [18]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [25]:
soup.select('#NewsAdContent')[0].text

'\n\t\t\t\t  SK그룹의 사회공헌은 일시적·시혜적(施惠的) 접근을 넘어서 나눔을 통해 구조적 문제점을 해결하자는 원칙 아래 운용되고 있다. 이에 따라 관련 프로그램들도 긴 호흡을 갖고 장기적으로 진행된다. \n\nSK의 사회공헌 슬로건은 ‘사회적 가치 창출을 통한 이해관계자와의 더 큰 행복 나눔’으로 요약된다. 최태원 회장은 ‘돈 버는 것만이 기업의 목적이 아니다’는 최종현 선대 회장의 사회공헌 철학을 ‘사회적 가치 추구’라는 화두로 계승·발전시켰다. 사회적 가치를 창출하는 것은 지속 가능한 기업을 만들기 위한 필수조건일 뿐 아니라, 기업이 더 크게 성장하기 위한 전제조건이라고 본다. \n\n특히 최 회장은 “사회적 가치 창출은 기존 시장과 고객을 놓고 서로 뺏거나 빼앗기는 제로섬(Zero-Sum) 게임이 아니라, 다양한 시장 플레이어와 함께 성장하며 새로운 시장을 만들어 나갈 수 있는 혁신적인 경영 전략”이라고 강조하고 있다. 최 회장은 사회적 가치 추구를 통한 신(新)경영전략의 3대 방법론으로 △경제적 가치와 사회적 가치를 동시에 추구하는 ‘DBL(Double Bottom Line) 경영’ △기업의 유무형 자산을 사회적 가치 창출 인프라로 활용하는 ‘공유 인프라’ △사회적 가치 창출 전문가와 협력하는 ‘사회적 기업 생태계 조성’을 제시하고 있다. SK그룹의 사회공헌은 이 같은 3대 방법론에 따라 이뤄진다.\n\n우선 DBL 경영은 기업이 전통적 개념의 경제적 가치에만 매몰되지 않고, 어떻게 하면 더 많은 경제적 가치와 사회적 가치의 ‘총합’을 만들어 낼 수 있을 것인가를 고민해야 한다는 경영 철학이다. 경제적 가치와 사회적 가치가 두 개의 핵심(Bottom Line)이 되는 것이다. 이런 원칙에 따라서 경영을 하면 비즈니스 모델을 바라보는 시각부터 바뀌게 되고, 그 결과 사회적 가치 창출을 통해 기업을 둘러싼 모든 이해관계자의 행복을 더 크게 키울 수 있다는 게 DBL 경영의 발상이다. SK는 올해 이를 그룹 전체 차원으로 확장, 주요 계열사 정관